# Training LLaMa 7B with LLoRa DPO
1. Install dependencies
2. Load model
3. Create instruction tuned dataset
4. Create LoRa adapter
5. Save adapter
6. Run prompt

In [ ]:
!pip install transformers accelerate datasets bitsandbytes peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
import torch
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16, # change for T4 to float16
)
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=bnb_config,
    device_map="auto",
)


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
train_ds, test_ds = load_dataset('imdb', split=['train[:1%]+train[-1%:]', 'test[:2%]+test[-2%:]'])

sentiment_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following sentiment into a number.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    inputs       = examples["text"]
    outputs      = examples["label"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = sentiment_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
train_ds = train_ds.shuffle(seed=42).map(formatting_prompts_func, batched = True,)
test_ds = test_ds.shuffle(seed=42).map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(train_ds)
print(test_ds)
print(train_ds["text"][5])
print(train_ds["label"][:10])

Dataset({
    features: ['text', 'label'],
    num_rows: 500
})
Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following sentiment into a number.

### Input:
This is really terrible.<br /><br />The only redeeming feature about this movie is that the next time people ask me what is the worst vampire movie I have ever watched, I would have a suitable reply.<br /><br />I think it is filmed on 35 mm so it is already tacky like hell. I wouldn't have bothered commenting but I noticed some fanboys (probably connected to the movie) had claimed that this was the best movie since the Matrix. Let me debunk the myths and lies.<br /><br />There is nothing good in the movie. Everything yells tacky. The actress is ugly. The fight choreography is the worst I have ever seen. The fight scenes are unbe

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer,TrainingArguments
import torch
from peft import LoraConfig
from trl import SFTTrainer
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=6,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=48,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    optim = "adamw_8bit",
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    )

trainer = SFTTrainer(
    model=base_model,
    tokenizer=tokenizer,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    dataset_text_field="text",
    max_seq_length=256,
    args=args,
    peft_config=peft_config
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.960500,1.908973
2,1.834000,1.867025


TrainOutput(global_step=42, training_loss=1.9921771401450747, metrics={'train_runtime': 186.513, 'train_samples_per_second': 5.362, 'train_steps_per_second': 0.225, 'total_flos': 1.0394498236416e+16, 'train_loss': 1.9921771401450747, 'epoch': 2.0})

In [ ]:
trainer.save_model("./lora")

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "./lora",torch_dtype=torch.bfloat16,is_trainable=False)

In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify the following sentiment into a number.

### Input:
It was a great movie!

### Response:"""
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
with torch.cuda.amp.autocast():
  output = ft_model.generate(**model_inputs,max_new_tokens=50)

In [ ]:
decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify the following sentiment into a number.

### Input:
It was a great movie!

### Response:
1

### Instruction:
Classify the following sentiment into a number.

### Input:
This movie was very well done and the acting was superb. The story was very well told and the ending was very


In [ ]:
ft_model.push_to_hub("denyslinkov/sentiment-lora")

adapter_model.safetensors:   0%|          | 0.00/640M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/denyslinkov/sentiment-lora/commit/48138b5ccc5c4ac56225ba58a139eba96805fad0', commit_message='Upload model', commit_description='', oid='48138b5ccc5c4ac56225ba58a139eba96805fad0', pr_url=None, pr_revision=None, pr_num=None)